## Objective: To identify genotype-phenotype trait association in Rice
### Develop a workflow to identify genes indirectly associated with rice traits (Grain Size, Grain number etc) using EKP and visualize them in an interactive knowledge graph.

In [1]:
library(dplyr)
library(tidyr)
library(sqldf)
library(splitstackshape)
library(stringr)
library(compare)



Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: gsubfn
Loading required package: proto
Loading required package: RSQLite
Loading required package: data.table
------------------------------------------------------------------------------
data.table + dplyr code now lives in dtplyr.
Please library(dtplyr)!
------------------------------------------------------------------------------

Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last


Attaching package: ‘compare’

The following object is masked from ‘package:base’:

    isTRUE



#### load EKP API

In [29]:
source("..//src/EuretosInfrastructure.R")

Retrieving page 0
Retrieving page 1
Retrieving page 2
Retrieving page 3
Retrieving page 4
Retrieving page 5
Retrieving page 6
Retrieving page 7
Retrieving page 8
Retrieving page 9
Retrieving page 10
Retrieving page 11


In [3]:
#### qtaro.abr.affrc.go.jp/qtab/table
setwd("~/odex4all_usecases/ODEX4all-UseCases/Bayer/data")

#### Data downloaded from QTARO database located at 
#### qtaro.abr.affrc.go.jp/qtab/table
#### This file can be changed to other gene symbols !

In [4]:
rice_genes <-read.csv("GeneInformationTable_Qtaro.csv",header=TRUE)

##### Select only morphological trait as these are associated with concept ids are dynamic (snapsnot date: 08-05-2017)
##### "grain size" (EKP concept id : 5899980)
##### "grain thickness" (EKP concept id  :5900661)
##### "grain number" (EKP concept id (rice specific) :4343608)
##### "kernel number" (EKP concept id:5900190)
##### "GRNB" (EKP concept:5900394)
##### "fruit number" (EKP concept:5900077)
##### "grain number per plant" (EKP concept (exact): 5900828)
##### "GN" (EKP concept:(vague many hits within EKP))

In [5]:
rice_genes <- select(rice_genes,locus_id,character_major)  
rice_genes <- filter(rice_genes, character_major == "Morphological trait")
rice_genes <- tolower(as.character(rice_genes[,"locus_id"]))
rice_genes <- unique(rice_genes)
rice_genes <- rice_genes[!is.na(rice_genes)]
rice_genes <-rice_genes[rice_genes != "-"]

In [42]:
head(rice_genes,n=10)

[1] "os01g0757200" "os02g0630300" "os04g0522500" "os09g0459200" "os02g0559800"
 [6] "os02g0560200" "os02g0560600" "os02g0561000" "os02g0561400" "os02g0561800"

## Step 1a : Get the starting concept identifiers

## Step 1b: Get the ending concept identifiers for various traits as represented

In [11]:
traits<-c("TO:0000590","TO:0000382","TO:0000396","TO:0000397","TO:0000734","TO:0000402","TO:0002759","TO:0000447")

In [12]:
### Trait ekp ids 
end<-NULL
for (i in 1:length(traits)){
  tmp <- getTraitEKPID(traits[i])
  tmpContent<-cbind(traits[i],tmp)
  end<-rbind(end,tmpContent)
}
end<-end[,c(2,3,4)]
colnames(end)<-c("TOid","TOEKPid","TOContentName")

head(end)



TOid,TOEKPid,TOContentName
TO:0000590,5899973,dehulled grain weight
TO:0000382,5900098,1000-seed weight
TO:0000396,5900965,grain yield trait
TO:0000397,5899980,grain size
TO:0000734,5900194,grain length
TO:0000402,5899965,grain width


### Step 2: For the traits that are not found get neighbours of them an create a file NeighbouringTrainEKPid

In [45]:
### Now get the neighbours of traits because there have no indirect relations between genes and traits that can be found within EKP
neighbours<-NULL
for (i in 1:length(end)){
  tmp <- unlist(getNeighbours(end[i]))
  tmp<-tmp[which(names(unlist(tmp))%in% "content.neighbour.id"==TRUE)]
  addEKPId<-cbind(end[i],tmp)
  neighbours<-rbind(neighbours,addEKPId)  
}
colnames(neighbours)<-c("TOEKPid","NeighbourEKPid")
rownames(neighbours)<-NULL
write.csv(neighbours,file="NeighbouringTraitEKPid.csv",row.names = FALSE)

ERROR: Error in rbind(neighbours, addEKPId): number of columns of matrices must match (see arg 2)


### Step 3: Get Indirect relationships between "rice genes"(start) and "Trait"(end)
#### WARNING: This takes really long time

In [ ]:
### Gets the JSON objects from EKP API
system("bash Get_TO:0000396.sh")
system("bash Get_TO:0002759.sh")
system("bash Get_TO:0000447.sh")
##################################################


#### load the file from the disk


In [22]:
### Read the JSON objects just created
document1 <- fromJSON(txt="int_TO:0000396.json",flatten=TRUE)$content
document2 <- fromJSON(txt="int_TO:0002759.json",flatten=TRUE)$content
document3 <- fromJSON(txt="int_TO:0000447.json",flatten=TRUE)$content


In [23]:
### Combine the documents together
genes2Traits <- list(document1,document2,document3)
df<-fromJSON(toJSON(genes2Traits),flatten=TRUE)
do.call(rbind,df) %>% as.data.frame -> genes2Traits


In [24]:
head(genes2Traits)

concepts,relationships,score
"7190948 , 5900394 , 5900965 , os07g0153600 (oryza sativa japonica), grain number , grain yield trait , Genes & Molecular Sequences , Physiology , Physiology , T028 , T040 , T040 , oryza sativa japonica","7190948 , 5900394 , 5900394 , 5900965 , 149947379, 149948843, 232226800, 214510142, 232227983, 10773543 , 10773540",17
"3942413 , 5062125 , 5900965 , loc4335790 (oryza sativa japonica) , beta-fructofuranosidase, insoluble isoenzyme 2 (oryza sativa subsp. japonica), grain yield trait , Genes & Molecular Sequences , Chemicals & Drugs , Physiology , T028 , T116 , T040 , oryza sativa japonica , oryza sativa subsp. japonica , oryza sativa subsp. indica","3942413 , 5062125 , 5062125 , 5900965 , 33276979 , 13981829 , 184887463, 182649453, 10773605 , 10773543",10
"3937158 , 5900772 , 5900965 , loc4347031 (oryza sativa japonica), grain yield per plant , grain yield trait , Genes & Molecular Sequences , Physiology , Physiology , T028 , T040 , T040 , oryza sativa japonica","3937158 , 5900772 , 5900772 , 5900965 , 13981962 , 149947923, 182653832, 232227184, 10773543 , 10773540",10
"3942239 , 5900828 , 5900394 , loc4336116 (oryza sativa japonica), grain number per plant , grain number , Genes & Molecular Sequences , Physiology , Physiology , T028 , T040 , T040 , oryza sativa japonica","3942239 , 5900828 , 5900828 , 5900394 , 149947366, 149947906, 232226791, 212140844, 232227168, 10773543 , 10773540",17
"3942413 , 5900965 , 5900394 , loc4335790 (oryza sativa japonica), grain yield trait , grain number , Genes & Molecular Sequences , Physiology , Physiology , T028 , T040 , T040 , oryza sativa japonica","3942413 , 5900965 , 5900965 , 5900394 , 13982456 , 149948843, 182655606, 232227983, 10773543 , 10773540",10
"3940353 , 5900965 , 5900394 , loc4324691 (oryza sativa japonica), grain yield trait , grain number , Genes & Molecular Sequences , Physiology , Physiology , T028 , T040 , T040 , oryza sativa japonica","3940353 , 5900965 , 5900965 , 5900394 , 13981952 , 149948843, 182653814, 232227983, 10773543 , 10773540",10


### Now call get indirect relationships for remaining traits (proxy for their neighbours)

In [ ]:
system ("bash Get_TO:0000590_TO:0000382_TO:0000397_TO:0000734_TO:0000402.sh")


In [39]:

genes2TraitNeighbour <- fromJSON(txt="int_TO:0000590_TO:0000382_TO:0000397_TO:0000734_TO:0000402.json")$content


In [31]:
head(genes2TraitNeighbour$relationships)

concept0Id,concept1Id,tripleIds,publicationIds,predicateIds
7191380,3018711,58958754,,10773565
3018711,5900111,598586,165079246,10773565
concept0Id,concept1Id,tripleIds,publicationIds,predicateIds
3939406,443490,58657858,204100383,10773588
443490,3940353,58581520,204087844,10773588
concept0Id,concept1Id,tripleIds,publicationIds,predicateIds
3939406,443490,58657858,204100383,10773588
443490,3941570,59715080,204261448,10773588
concept0Id,concept1Id,tripleIds,publicationIds,predicateIds
3939406,2249148,58657850,204100383,10773588


### Combine all traits

In [25]:
overallTraitRelations<-rbind(genes2Traits,genes2TraitNeighbour)

In [32]:
head(overallTraitRelations$relationships)

concept0Id,concept1Id,tripleIds,publicationIds,predicateIds
7190948,5900394,149947379,"232226800, 214510142",10773543
5900394,5900965,149948843,232227983,10773540
concept0Id,concept1Id,tripleIds,publicationIds,predicateIds
3942413,5062125,33276979,184887463,10773605
5062125,5900965,13981829,182649453,10773543
concept0Id,concept1Id,tripleIds,publicationIds,predicateIds
3937158,5900772,13981962,182653832,10773543
5900772,5900965,149947923,232227184,10773540
concept0Id,concept1Id,tripleIds,publicationIds,predicateIds
3942239,5900828,149947366,"232226791, 212140844",10773543


### Formatting and data cleaning

In [33]:
dfs<-as.matrix(getTableFromJson(overallTraitRelations))
dfs[,"Predicate"]<-str_replace_all(dfs[,"Predicate"], "[^[:alnum:]]","")
dfs[,"Predicate"]<-str_replace_all(dfs[,"Predicate"], "c","")
dfs[,"Publications"]<-str_replace_all(dfs[,"Publications"], "[^[:alnum:]]","")
dfs[,"Publications"]<-str_replace_all(dfs[,"Publications"], "c","")
dfs<- data.frame(dfs, stringsAsFactors=FALSE)

Warning message in cbind(unlist(dfs[, "Subject"]), unlist(dfs[, "Object"]), as.character.default(unlist(dfs[, :
“number of rows of result is not a multiple of vector length (arg 1)”

In [34]:
head(dfs)

Subject,Predicate,Object,Publications,Score
7190948,10773543,5900394,232226800,17
5900394,10773540,5900965,214510142,10
3942413,10773605,5062125,232227983,10
5062125,10773543,5900965,184887463,17
3937158,10773543,5900772,182649453,10
5900772,10773540,5900965,182653832,10


### Step 4: Map human redable triples from the reference database 
### reference list is collected from EKP

In [ ]:
pred<-read.csv("Reference_Predicate_List.csv",header=TRUE)
pred<-pred[,c(2,3)]
colnames(pred)<-c("pred","names")


subject_name<-getConceptName(dfs[,"Subject"])
dfs<-cbind(dfs,subject_name[,1])

object_name<-getConceptName(dfs[,"Object"])
dfs<-cbind(dfs,object_name[,1])

predicate_name<-sqldf('select * from dfs left join pred on pred.pred=dfs.Predicate')

pbs<-getPubMedId(dfs$Publications)

tripleName<-cbind(subject_name,as.character(predicate_name[,"names"]),object_name,pbs,dfs[,"Score"])
colnames(tripleName)<-c("Subject","Predicate","Object","Provenance","Score")

write.table(tripleName,file="~/ODEX4all-UseCases/Bayer/data/Results_TO:0000396_TO:0002759_TO:0000447_TO:0000590_TO:0000382_TO:0000397_TO:0000734_TO:0000402.csv",sep=",",row.names = FALSE)
